# Reestructuración y limpieza de datos

Este cuaderno se encarga de la reestructuración y limpieza de los datos. Los pasos que sigue son los siguientes:

1. **Lectura de datos**: Se importan las bibliotecas necesarias y se cargan los dataframes previamente creados, tanto el que contiene los datos originales como el que contiene los datos editados.


2. **Organización del DataFrame**: Se realiza una serie de transformaciones en los dataframes para organizarlos y hacerlos más manejables. Esto incluye la unificación de las columnas de multiindex en un solo index, el reemplazo de los espacios en blanco por guiones bajos en los nombres de las columnas, la renombración de algunas columnas para una mejor interpretación y la conversión de todos los nombres de las columnas a mayúsculas.


3. **Transformación y limpieza de Datos**: Se convierten los datos en las columnas 'FECHA' y 'HORA' a sus respectivos tipos de datos. Además, se implementa una función para eliminar la hora cero de los datframes, puesto que, se cuenta con la hora 24. Durante esta, notamos que había datos mal escritos, por ejemplo, se encontraron valores como 2,.4 o 4. o 2,4* o 2, 4. Para resolver esta situación, se eliminaron los símbolos, espacios y otros caracteres que interferían con la conversión del dato en un número válido, ya sea flotante o entero. Aquellos datos que no pudieron ser convertidos se registraron como NaN y luego se convierten a tipo float. Por indicaciones dadas por el ingeniero Mauricio los datos de dosis de coagulante que tuvieran decimales se deben redondear hacia abajo al entero más cercano.


4. **Evaluación del relleno manual**: Siguiendo las especificaciones proporcionadas por el ingeniero Mauricio, se llevó a cabo un rellenado manual en los archivos de Excel correspondientes a los años 2013 al 2020. Si había espacios entre datos de un mismo día, se completaba con la dosis de coagulante anterior más cercana, siempre que los datos de agua cruda fueran muy similares. En caso contrario, si la turbiedad era menor o igual a 3, se rellenaba con cero. Además, se estableció que si un día completo estaba vacío y el valor de turbiedad cruda era menor o igual a 3, la dosis de coagulante debía ser cero. Para los años 2021 y 2022, el ingeniero Mauricio indicó que se debería coagular siempre en las noches y coagular durante el día únicamente si la turbiedad era mayor a 3. Por lo tanto, se creó una función `reemplazar_nan` para evaluar y rectificar el llenado manual realizado previamente y se aplicó el proceso de rellenado para los años 2021 y 2022.


5. **Creación de CSV:** Al final del proceso, se guardan los dataframes reestructurados y limpios en archivos CSV para su posterior análisis.

## 1. Lectura de datos

In [1]:
# Importación de bibliotecas necesarias
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import math

In [2]:
# Lectura de dataframes
df = pd.read_csv('../data/dataframe.csv',sep=',', encoding='latin-1', header=[0, 1], index_col=0, low_memory=False)
# df_edit = pd.read_csv('../data_edit/dataframe_edit.csv',sep=',', encoding='latin-1', header=[0, 1], index_col=0, low_memory=False)

## 2. Organización del DataFrame

In [3]:
def df_order(df):
    # Une columnas multi-index en un solo índice con guión bajo ('_').
    df.columns = df.columns.map('_'.join)

    # Reemplaza espacios en nombres de columnas por guiones bajos.
    df.columns = [col.replace(' ', '_') for col in df.columns]

    # Renombra algunas columnas para una mejor comprensión.
    df = df.rename(columns={'FECHA_Unnamed:_1_level_1': 'FECHA', 'HORA_Unnamed:_2_level_1': 'HORA','CAUDAL_Entr_l/s':'CAUDAL','Cal_1ria_Kg.':'Cal_1ria_Kg'})

    # Convierte todos los nombres de columnas a mayúsculas.
    df.columns = df.columns.str.upper()
    
    return df


In [4]:
# Aplica la función df_order a ambos DataFrames.
df = df_order(df)
# df_edit = df_order(df_edit)

## 3. Transformación y limpieza de Datos 

In [5]:
# Tranformación de columnas 'CAUDAL', 'HORA' y 'FECHA'
def convert_data(df):
    # Conversión de datos de FECHA y HORA
    # Convierte 'FECHA' a formato datetime y 'HORA' a int
    df['FECHA'] = pd.to_datetime(df['FECHA'], format='%d/%m/%Y')
    df['HORA'] = df['HORA'].astype(int)
    
    return df

In [6]:
# Aplica la función convert_data en ambos DataFrames
df = convert_data(df)
# df_edit = convert_data(df_edit)

## 4. Limpieza

In [7]:
# Eliminación de hora cero
def del_hour_zero(df):
    # Filtra las filas donde 'HORA' es distinto de cero
    filtro_hora = df['HORA'] != 0
    df = df[filtro_hora]
    
    # Resetea el índice del DataFrame
    df = df.reset_index(drop=True)
    return df

In [8]:
# Aplica la función del_hour_zero en ambos DataFrames
df = del_hour_zero(df)
# df_edit = del_hour_zero(df_edit)

In [9]:
# Limpieza de celdas
def cell_clean(cell):
    # Reemplaza 'NO HAY AGUA' por 0
    if isinstance(cell, str) and cell.strip().upper() == 'NO HAY AGUA':
        return 0

    # Verifica si la celda contiene un valor NaN. Si la celda es NaN, la retorna tal cual
    if pd.isna(cell):
        return cell

    # Buscar y eliminar caracteres no deseados (solo conserva números y puntos)
    cell = cell.replace(',', '.')
    cell_cleaned = re.sub(r'[^0-9.]', '', str(cell))

    # Convertir la celda a un número si es posible, de lo contrario, devuelve un NaN
    try:
        return float(cell_cleaned) if '.' in cell_cleaned else int(cell_cleaned)
    except ValueError:
        return np.nan

In [10]:
# Aplicación de función anterior
def apply_clean(df):
    columns_object = df.select_dtypes(include='object').columns
    df[columns_object] = df[columns_object].applymap(cell_clean)
    
    # Transformación de columnas limpias a float
    df[columns_object] = df[columns_object].astype(float)
    return df

In [11]:
# Aplicación de función apply_clean en ambos Dataframes
df = apply_clean(df)
# df_edit = apply_clean(df_edit)

In [12]:
# Redondeo hacia abajo al entero más cercano
def round_coagulant(df):
    df['COAGULANTE_DOSIS'] = df['COAGULANTE_DOSIS'].apply(lambda x: math.floor(x) if pd.notnull(x) else x)
    return df

In [13]:
# df_edit.loc[df['AGUA_TRATADA_P.H'] > 14, 'AGUA_TRATADA_P.H'] = df_edit.loc[df['AGUA_TRATADA_P.H'] > 14, 'AGUA_TRATADA_P.H'] / 10

In [14]:
# Aplicación de función round_coagulant en ambos Dataframes
df = round_coagulant(df)
# df_edit = round_coagulant(df_edit)

## 5. Evaluación del relleno manual

In [15]:
def rectify_coagulant_dosis(df):
    # Crea un DataFrame para los años 2021 y 2022
    df_2021_2022 = df[(df['FECHA'].dt.year == 2021) | (df['FECHA'].dt.year == 2022)]

    # Si la turbidez es <= 3 y la hora está entre las 8 y las 20, entonces la dosis de coagulante es 0
    condition = (df_2021_2022['COAGULANTE_DOSIS'].isnull()) & (df_2021_2022['AGUA_CRUDA_NTU'] < 3) & (df_2021_2022['HORA'].between(8, 20))
    df_2021_2022.loc[condition, 'COAGULANTE_DOSIS'] = 0

    # Crea un DataFrame para los años distintos a 2021 y 2022
    df_other_years = df[(df['FECHA'].dt.year != 2021) & (df['FECHA'].dt.year != 2022)]
    condition = (df_other_years['COAGULANTE_DOSIS'].isnull()) & (df_other_years['AGUA_CRUDA_NTU'] < 3)
    df_other_years.loc[condition, 'COAGULANTE_DOSIS'] = 0

    # Combina los dos dataframes
    df = pd.concat([df_other_years,df_2021_2022])
    
    return df

In [16]:
# Aplicación de función relleno sólo al dataframe editado
df_edit = rectify_coagulant_dosis(df)

## 6. Creación de CSV

In [17]:
# Creación de CSV
df.to_csv('../data/dataframe_clean.csv', sep=',')
df_edit.to_csv('../data_edit/dataframe_edit_clean.csv', sep=',')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87648 entries, 0 to 87647
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   FECHA                     87648 non-null  datetime64[ns]
 1   HORA                      87648 non-null  int32         
 2   CAUDAL                    87380 non-null  float64       
 3   CAL_1RIA_KG               29 non-null     float64       
 4   CAL_1RIA_DOSIS            587 non-null    float64       
 5   AGUA_CRUDA_P.H            86715 non-null  float64       
 6   AGUA_CRUDA_COLOR          81227 non-null  float64       
 7   AGUA_CRUDA_NTU            86984 non-null  float64       
 8   AGUA_CRUDA_ALCALINIDAD    68371 non-null  float64       
 9   AGUA_CRUDA_CONDUCTIVIDAD  60089 non-null  float64       
 10  COAGULANTE_GRANULADO      5897 non-null   float64       
 11  COAGULANTE_LIQUIDO        135 non-null    float64       
 12  COAGULANTE_DOSIS  

In [19]:
df_edit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87648 entries, 0 to 87647
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   FECHA                     87648 non-null  datetime64[ns]
 1   HORA                      87648 non-null  int32         
 2   CAUDAL                    87380 non-null  float64       
 3   CAL_1RIA_KG               29 non-null     float64       
 4   CAL_1RIA_DOSIS            587 non-null    float64       
 5   AGUA_CRUDA_P.H            86715 non-null  float64       
 6   AGUA_CRUDA_COLOR          81227 non-null  float64       
 7   AGUA_CRUDA_NTU            86984 non-null  float64       
 8   AGUA_CRUDA_ALCALINIDAD    68371 non-null  float64       
 9   AGUA_CRUDA_CONDUCTIVIDAD  60089 non-null  float64       
 10  COAGULANTE_GRANULADO      5897 non-null   float64       
 11  COAGULANTE_LIQUIDO        135 non-null    float64       
 12  COAGULANTE_DOSIS  